In [1]:
#Group project!



We've been hired by a new fizzy seltzer brand called Brian's Booze. The company has hired us as Data Scientists to determine which countries have the highest alcohol consumption so that their marketing team can target those countries to build/sell their brand in. It is our job to use different lifestyle variables that will help predict which countries have the highest alcohol consumption per capita. 

We will cluster alcohol consumption into low, medium, and high categories. 

In [2]:
# Load libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
#let import all out data
#Literacy Rate Data
alc_df = pd.read_csv("/Users/alexandratorti/DS-3001/DS-3001/Final_Project/alcohol.csv")
df_alc = pd.melt(alc_df, id_vars=['Country Name', 'Country Code', 'Series Name', "Series Code"], var_name='Year', value_name='Alcohol Consumption')
columns_to_drop = ['Series Name', 'Series Code', 'Country Code']
df_alc = df_alc.drop(columns=columns_to_drop)
df_alc.head()

,Country Name,Year,Alcohol Consumption
0,Afghanistan,2015 [YR2015],0.00874
1,Africa Eastern and Southern,2015 [YR2015],4.75979799686623
2,Africa Western and Central,2015 [YR2015],4.4374260825495
3,Albania,2015 [YR2015],5.03752
4,Algeria,2015 [YR2015],0.5844


In [4]:
#Literacy Rate Data
lit_df = pd.read_csv("/Users/alexandratorti/DS-3001/DS-3001/Final_Project/Literacy_rate_data.csv")
df_lit = pd.melt(lit_df, id_vars=['Country Name', 'Country Code', 'Series Name', "Series Code"], var_name='Year', value_name='Literacy_rate')
columns_to_drop = ['Series Name', 'Series Code', 'Country Code']
df_lit = df_lit.drop(columns=columns_to_drop)
df_lit.head()

,Country Name,Year,Literacy_rate
0,Afghanistan,2015 [YR2015],..
1,Albania,2015 [YR2015],..
2,Algeria,2015 [YR2015],..
3,American Samoa,2015 [YR2015],..
4,Andorra,2015 [YR2015],..


In [5]:
#Birth Rate Data
birth_df = pd.read_csv("/Users/alexandratorti/DS-3001/DS-3001/Final_Project/birthratedata.csv")
df_birth = pd.melt(birth_df, id_vars=['Country Name', 'Country Code', 'Series Name', "Series Code"], var_name='Year', value_name='birth_rate')
df_birth = df_birth.drop(columns=columns_to_drop)
df_birth.head()

,Country Name,Year,birth_rate
0,Afghanistan,2015 [YR2015],38.803
1,Albania,2015 [YR2015],11.718
2,Algeria,2015 [YR2015],25.405
3,American Samoa,2015 [YR2015],..
4,Andorra,2015 [YR2015],..


In [6]:
#Unemployment Rate Data
unem_df = pd.read_csv("/Users/alexandratorti/DS-3001/DS-3001/Final_Project/unemploymentdata.csv")
df_unem = pd.melt(unem_df, id_vars=['Country Name', 'Country Code', 'Series Name', "Series Code"], var_name='Year', value_name='unemployment_rate')
df_unem = df_unem.drop(columns=columns_to_drop)
df_unem.head()

,Country Name,Year,unemployment_rate
0,Afghanistan,2015 [YR2015],..
1,Albania,2015 [YR2015],17.19
2,Algeria,2015 [YR2015],11.21
3,American Samoa,2015 [YR2015],..
4,Andorra,2015 [YR2015],..


In [7]:
#GDP Data
gdp_df = pd.read_csv("/Users/alexandratorti/DS-3001/DS-3001/Final_Project/gdpdata.csv")
df_gdp = pd.melt(gdp_df, id_vars=['Country Name', 'Country Code', 'Series Name', "Series Code"], var_name='Year', value_name='GDP')
df_gdp = df_gdp.drop(columns=columns_to_drop)
df_gdp.head()

,Country Name,Year,GDP
0,Afghanistan,2015 [YR2015],592.476164793256
1,Albania,2015 [YR2015],3952.80358410846
2,Algeria,2015 [YR2015],4197.41997101868
3,American Samoa,2015 [YR2015],13101.5418159165
4,Andorra,2015 [YR2015],38885.5303237179


In [8]:
#Death Rate Data
death_df = pd.read_csv("/Users/alexandratorti/DS-3001/DS-3001/Final_Project/deathdata.csv")
df_death = pd.melt(death_df, id_vars=['Country Name', 'Country Code', 'Series Name', "Series Code"], var_name='Year', value_name='Death_rate')
df_death = df_death.drop(columns=columns_to_drop)
df_death.head()

,Country Name,Year,Death_rate
0,Afghanistan,2015 [YR2015],7.331
1,Albania,2015 [YR2015],7.947
2,Algeria,2015 [YR2015],4.437
3,American Samoa,2015 [YR2015],..
4,Andorra,2015 [YR2015],..


In [9]:
#Labor Force Participation Data
labor_df = pd.read_csv("/Users/alexandratorti/DS-3001/DS-3001/Final_Project/laborparticipation.csv")
df_labor = pd.melt(labor_df, id_vars=['Country Name', 'Country Code', 'Series Name', "Series Code"], var_name='Year', value_name='LF_rate')
df_labor = df_labor.drop(columns=columns_to_drop)
df_labor.head()

,Country Name,Year,LF_rate
0,Afghanistan,2015 [YR2015],..
1,Albania,2015 [YR2015],31.3
2,Algeria,2015 [YR2015],25.19
3,American Samoa,2015 [YR2015],..
4,Andorra,2015 [YR2015],..


In [10]:
#now its time to merge all out dataframes. 
# Merge DataFrames
merged_df = pd.merge(df_lit, df_birth, on=['Country Name', 'Year'])
merged_df = pd.merge(merged_df, df_unem, on=['Country Name', 'Year'])
merged_df = pd.merge(merged_df, df_gdp, on=['Country Name', 'Year'])
merged_df = pd.merge(merged_df, df_death, on=['Country Name', 'Year'])
merged_df = pd.merge(merged_df, df_labor, on=['Country Name', 'Year'])
merged_df = pd.merge(merged_df, df_alc, on=['Country Name', 'Year'])
merged_df.head()

,Country Name,Year,Literacy_rate,birth_rate,unemployment_rate,GDP,Death_rate,LF_rate,Alcohol Consumption
0,Afghanistan,2015 [YR2015],..,38.803,..,592.476164793256,7.331,..,0.00874
1,Albania,2015 [YR2015],..,11.718,17.19,3952.80358410846,7.947,31.3,5.03752
2,Algeria,2015 [YR2015],..,25.405,11.21,4197.41997101868,4.437,25.19,0.5844
3,American Samoa,2015 [YR2015],..,..,..,13101.5418159165,..,..,..
4,Andorra,2015 [YR2015],..,..,..,38885.5303237179,..,..,10.76055


## Drop variables that will not be needed or are duplicates

In [11]:
#lets take a look at our data and see what needs cleaning!
merged_df.head()

na_counts = merged_df.isna().sum()
print(na_counts)
# Check for NaN values in each row
rows_with_na = merged_df.isna().any(axis=1)

# Display rows with NaN values
print("Rows with NaN values:")
print(merged_df[rows_with_na])# there seems to be some errors when I merged the data frames that added a bunch of NaN values. Lets delete these extra rows because we will not need them

Country Name           0
Year                   0
Literacy_rate          0
birth_rate             0
unemployment_rate      0
GDP                    0
Death_rate             0
LF_rate                0
Alcohol Consumption    0
dtype: int64
Rows with NaN values:
Empty DataFrame
Columns: [Country Name, Year, Literacy_rate, birth_rate, unemployment_rate, GDP, Death_rate, LF_rate, Alcohol Consumption]
Index: []


In [12]:

# Drop rows with NaN values in column Country
df_cleaned = merged_df.dropna(subset=['Country Name'], how='any')

#DataFrame after dropping rows with NaN values in column Country Name
# print(df_cleaned)

na_counts = df_cleaned.isna().sum()
print(na_counts)


Country Name           0
Year                   0
Literacy_rate          0
birth_rate             0
unemployment_rate      0
GDP                    0
Death_rate             0
LF_rate                0
Alcohol Consumption    0
dtype: int64


In [26]:
#Need to drop .. values so that we can transform values to floats
df_cleaned.replace('..', np.nan, inplace=True)
df_cleaned.info()


#We need to change the columns that are objects to strings and floats
columns_to_convert = ['Literacy_rate', 'birth_rate', "unemployment_rate", "GDP", "Death_rate", "LF_rate", "Alcohol Consumption"]
df_cleaned[columns_to_convert] = df_cleaned[columns_to_convert].astype(float)


columns_to_convert2 = ['Country Name', 'Year']
df_cleaned[columns_to_convert2] = df_cleaned[columns_to_convert2].astype(str)

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country Name         432 non-null    object 
 1   Year                 432 non-null    object 
 2   Literacy_rate        65 non-null     float64
 3   birth_rate           427 non-null    float64
 4   unemployment_rate    247 non-null    float64
 5   GDP                  418 non-null    float64
 6   Death_rate           427 non-null    float64
 7   LF_rate              228 non-null    float64
 8   Alcohol Consumption  372 non-null    float64
dtypes: float64(7), object(2)
memory usage: 30.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country Name         432 non-null    object 
 1   Year                 432 non-null 

In [27]:
#For the first time we cluster, lets use 2019 data.
df = pd.DataFrame(df_cleaned)

# Specify the year you want to keep
target_year = '2019 [YR2019]'

# Use boolean indexing to filter the DataFrame
df_filtered = df[df['Year'] == target_year]

# Display the result
print(df_filtered)


              Country Name           Year  Literacy_rate  birth_rate  \
216            Afghanistan  2019 [YR2019]            NaN      36.466   
217                Albania  2019 [YR2019]            NaN      10.343   
218                Algeria  2019 [YR2019]            NaN      23.298   
219         American Samoa  2019 [YR2019]            NaN         NaN   
220                Andorra  2019 [YR2019]            NaN       7.000   
..                     ...            ...            ...         ...   
427  Virgin Islands (U.S.)  2019 [YR2019]            NaN      12.300   
428     West Bank and Gaza  2019 [YR2019]      97.378258      29.466   
429            Yemen, Rep.  2019 [YR2019]            NaN      31.778   
430                 Zambia  2019 [YR2019]            NaN      35.462   
431               Zimbabwe  2019 [YR2019]            NaN      31.518   

     unemployment_rate           GDP  Death_rate  LF_rate  Alcohol Consumption  
216                NaN    500.522981       6.791      

In [ ]:
#lets drop the values that dont have target variable numbers

In [28]:
#Now lets normalize the data!
numeric_columns = df_filtered.select_dtypes("float", "int").columns
print(numeric_columns)
df_filtered[numeric_columns]= MinMaxScaler().fit_transform(df_filtered[numeric_columns])


Index(['Literacy_rate', 'birth_rate', 'unemployment_rate', 'GDP', 'Death_rate',
       'LF_rate', 'Alcohol Consumption'],
      dtype='object')


/var/folders/55/4w5l1r5x0618j7pxmq4kdgmr0000gn/T/ipykernel_61728/2033875034.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[numeric_columns]= MinMaxScaler().fit_transform(df_filtered[numeric_columns])


In [ ]:
df_cleaned = df_cleaned['Year', 'Literacy-rate', 'birth_rate, unemployment']

## Run the clustering algo with your best guess for K

## View the results


## Create a visualization of the results with 2 or 3 variables that you think will best differentiate the clusters

## Evaluate the quality of the clustering using total variance explained and silhouette scores

## Determine the ideal number of clusters using the elbow method and the silhouette coefficient

## Visualize the results of the elbow method

## Use the recommended number of cluster (assuming it's different) to retrain your model and visualize the results

## Once again evaluate the quality of the clustering using total variance explained and silhouette scores

## Use the model to select countries for company to consider